In [22]:
pip install spotify

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey, UniqueConstraint
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from spotipy.oauth2 import SpotifyClientCredentials
from sqlalchemy import select
import requests
import random
import spotipy

In [29]:
MY_OWN_API = '+ZTJlCkxT1OvdkpNrYq31Q==IPWK1QtYmMeFY8J3'
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''

# Interaction with the API

In [12]:
categories = {
    "Neutral": ["life", "inspirational", "hope", "future", "imagination"],
    "Joy": ["happiness", "love", "friendship", "funny", "humor", "success", "great", "good"],
    "Sadness": ["alone", "failure", "death"],
    "Fear": ["fear", "courage"],
    "Surprise": ["amazing", "cool"],
    "Anger": ["anger", "jealousy"],
    "Shame": ["failure"],
    "Disgust": ["anger"],
}


In [14]:
def get_quote_by_emotion(emotion):
    if emotion not in categories:
        print(f"No categories mapped for emotion: {emotion}")
        return None
    
    # we shuffle the categories to add randomness
    emotion_categories = categories[emotion]
    random.shuffle(emotion_categories)
    
    for category in emotion_categories:
        api_url = f'https://api.api-ninjas.com/v1/quotes?category={category}'
        response = requests.get(api_url, headers={'X-Api-Key': MY_OWN_API})
        if response.status_code == requests.codes.ok:
            data = response.json()
            if data:
                quote_text = data[0]['quote']
                author = data[0]['author']
                return {
                    'quote': quote_text,
                    'author': author,
                    'category': category,
                    'emotion': emotion
                }
        else:
            print(f"Error: {response.status_code} {response.text}")
    print(f"No quotes found for emotion: {emotion}")
    return None

In [57]:
# this is just a test, the real emotion should come from my AI model
emotion_detected = 'Joy'  
quote_data = get_quote_by_emotion(emotion_detected)
if quote_data:
    print(f"Recommended Quote for {emotion_detected}:")
    print(f"\"{quote_data['quote']}\" - {quote_data['author']}")
else:
    print("Could not find a quote.")

Recommended Quote for Joy:
"Sometimes, comics will make the observation that it's not jokes that are funny, it's characters that are funny. And isn't that true! That's why I always kill jokes. I'm terrible at them, because I get the joke right, but I can't get the character right, and it just goes down like a lead balloon." - David Mitchell


# Interaction with the DB

In [45]:
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey, UniqueConstraint
from sqlalchemy.orm import sessionmaker, relationship, declarative_base

# SQLAlchemy 2.0's declarative_base is imported differently
Base = declarative_base()

# Define the Quote table
class Quote(Base):
    __tablename__ = 'quotes'
    quote_id = Column(Integer, primary_key=True)
    text = Column(Text, unique=True)  # The quote text should be unique
    author = Column(String)
    category = Column(String)
    emotion = Column(String)
    feedbacks = relationship('UserFeedback', back_populates='quote')  # Relationship to feedback

# Define the User table
class User(Base):
    __tablename__ = 'users'
    user_id = Column(Integer, primary_key=True)
    username = Column(String, unique=True)  # Unique username for identification
    feedbacks = relationship('UserFeedback', back_populates='user')

# Define the UserFeedback table for thumbs up/down
class UserFeedback(Base):
    __tablename__ = 'user_feedback'
    feedback_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.user_id'))
    quote_id = Column(Integer, ForeignKey('quotes.quote_id'))
    feedback = Column(String)  # 'thumbs_up' or 'thumbs_down'

    user = relationship('User', back_populates='feedbacks')
    quote = relationship('Quote', back_populates='feedbacks')

    __table_args__ = (UniqueConstraint('user_id', 'quote_id', name='_user_quote_uc'),)

In [46]:
# Create the SQLite database
engine = create_engine('sqlite:///quotes.db')

# Create the session
Session = sessionmaker(bind=engine)
session = Session()

# Create all tables in the database
Base.metadata.create_all(engine)

In [47]:
def save_quote_to_db(quote_data):
    # Check if the quote already exists in the database
    existing_quote = session.query(Quote).filter_by(text=quote_data['quote']).first()
    
    # If the quote doesn't exist, create a new entry
    if not existing_quote:
        new_quote = Quote(
            text=quote_data['quote'],
            author=quote_data['author'],
            category=quote_data['category'],
            emotion=quote_data['emotion']
        )
        session.add(new_quote)
        session.commit()
        return new_quote
    return existing_quote  # If the quote already exists, return the existing record

In [48]:
def save_user_feedback(user_id, quote_id, feedback):
    # Check if the user has already given feedback for this quote
    existing_feedback = session.query(UserFeedback).filter_by(user_id=user_id, quote_id=quote_id).first()
    
    # If the feedback doesn't exist, create a new feedback entry
    if not existing_feedback:
        new_feedback = UserFeedback(
            user_id=user_id,
            quote_id=quote_id,
            feedback=feedback  # e.g., 'thumbs_up' or 'thumbs_down'
        )
        session.add(new_feedback)
        session.commit()
    else:
        # If feedback already exists, update the feedback
        existing_feedback.feedback = feedback
        session.commit()

In [51]:
def get_quote_for_user(emotion, user_id):
    # Step 1: Try to get quotes the user liked for this emotion
    liked_quotes = session.query(Quote).join(UserFeedback).filter(
        UserFeedback.user_id == user_id,
        UserFeedback.feedback == 'thumbs_up',
        Quote.emotion == emotion
    ).all()

    # If the user has liked quotes, prioritize showing those first
    if liked_quotes:
        return random.choice(liked_quotes)

    # Step 2: Get quotes the user hasn't seen yet (unseen quotes)
    # Explicitly turn the subquery into a select() statement
    seen_quote_ids = select(UserFeedback.quote_id).filter_by(user_id=user_id)
    
    unseen_quotes = session.query(Quote).filter(
        Quote.emotion == emotion,
        ~Quote.quote_id.in_(seen_quote_ids)
    ).all()

    # If there are unseen quotes, return one randomly
    if unseen_quotes:
        return random.choice(unseen_quotes)

    # Step 3: If all quotes have been seen, return any quote for the emotion
    any_quotes = session.query(Quote).filter_by(emotion=emotion).all()
    if any_quotes:
        return random.choice(any_quotes)

    # Step 4: If no quotes are found in the database, fetch from the API
    quote_data = get_quote_by_emotion(emotion)
    if quote_data:
        saved_quote = save_quote_to_db(quote_data)
        return saved_quote

    return None  # If no quote is found in any step


# Recommending songs - not done

In [32]:
song_categories = {
    "Neutral": ["playlist_id_neutral"],
    "Joy": ["playlist_id_joy"],
    "Sadness": ["playlist_id_sadness"],
    "Fear": ["playlist_id_fear"],
    "Surprise": ["playlist_id_surprise"],
    "Anger": ["playlist_id_anger"],
    "Shame": ["playlist_id_shame"],
    "Disgust": ["playlist_id_disgust"]
}

In [ ]:
def get_song_by_emotion(motion):
    playlist_ids = song_categories.get(emotion, [])
    
    if not playlist_ids:
        print(f"No playlists mapped for emotion: {emotion}")
        return None
    
    # Choose a random playlist for variety
    playlist_id = random.choice(playlist_ids)
    
    # Fetch tracks from the selected playlist
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    
    # If tracks are found, choose one randomly and return its details
    if tracks:
        track = random.choice(tracks)
        track_info = {
            'song_name': track['track']['name'],
            'artist': track['track']['artists'][0]['name'],
            'url': track['track']['external_urls']['spotify']
        }
        return track_info
    else:
        print(f"No tracks found in playlist: {playlist_id}")
        return None

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Testing emotions

In [56]:
# Step 1: User requests a quote for 'Joy'
emotion_detected = 'Joy'
user_id = 1

quote = get_quote_for_user(emotion_detected, user_id)

if quote:
    print(f"Quote for {emotion_detected}: \"{quote.text}\" - {quote.author}")
    
    # Simulate feedback (user likes the quote)
    user_feedback = 'thumbs_up'
    save_user_feedback(user_id, quote.quote_id, user_feedback)
    print(f"Feedback ('{user_feedback}') saved for user {user_id} on quote ID {quote.quote_id}.")
else:
    print("No quote found.")

# Step 2: User requests another quote for 'Joy' (system should prioritize liked quotes)
quote = get_quote_for_user(emotion_detected, user_id)

if quote:
    print(f"Quote for {emotion_detected}: \"{quote.text}\" - {quote.author}")
else:
    print("No quote found.")

Quote for Joy: "If I didn't try to eavesdrop on every bus ride I take or look for the humor when I go for a walk, I would just be depressed all the time." - Lynda Barry
Feedback ('thumbs_up') saved for user 1 on quote ID 1.
Quote for Joy: "If I didn't try to eavesdrop on every bus ride I take or look for the humor when I go for a walk, I would just be depressed all the time." - Lynda Barry


# API endpoints to implement

In [26]:
@app.route('/get_recommendations', methods=['POST']):
    # 1. we get username coming from front end
    # 2. call AI method to get an emotion
    # 3. querying by user username to get user_id
    # 4. getting our quote
    quote = get_quote_for_user(emotion, user_id)
    # 5. getting our song
    song = get_song_by_emotion(emotion)
    # 6. sending to the frontend
    response = {
        'quote': {
            'text': quote.text,
            'author': quote.author
            },
        'song': song
        }

SyntaxError: invalid syntax (4156581683.py, line 1)